In [114]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [164]:
# ClickHouse client execution function.
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

from clickhouse_driver import Client
from clickhouse_driver.errors import ServerException

SETTINGS={'max_partitions_per_insert_block': 10000}

def execute_df_to_ch(clickhouse_client, ch_table, max_fetch_rows, df, \
    planopos_company_weekly=False, truncate=False):
    """Execute sql on HANA database and insert into ClickHouse in max_fetch_rows batches.
    Args:
        clickhouse (clickhouse_driver.client.Client): instance of ClickHouse client
        ch_table (str): destination table in ClickHouse
        max_fetch_rows (int): how many rows to fetch from HANA per one batch.
        daily_sales_transform (bool): whether to do data transforms specific to sales_detail HANA table
    Returns:
        dict:
            row_number (int): total number of ingested rows
            truncated (bool): whether ch_table was truncated or not
    """

    ch_insert_sql = f"INSERT INTO {ch_table} VALUES"

    table_truncated = False
    row_number = 0
    start = 0
    end = max_fetch_rows

    while True:
        print(f"Fetching {row_number}~ rows...")
        result = df[start:end]
        print(f"dataframe fetched from {start}~{end} rows...")
        
        if len(result) == 0 and row_number == 0: raise Exception("Failed to fetch any data.")
        
        if planopos_company_weekly:
            payload = result.values.tolist()
        else:
            payload = result

        print(f"Inserting {len(payload)} rows into ClickHouse {ch_table}.")
        clickhouse_client.execute(ch_insert_sql, payload, settings=SETTINGS)
        row_number += len(payload)
        start += len(payload)
        end += len(payload)
        if len(payload) < max_fetch_rows:
            print("Reached the end of query result set.")
            print(f"Total number of rows: {row_number}")
            break
    
    return {"row_number": row_number, "truncated": table_truncated}

def get_clickhouse_client(environment="commercial"):
    """Connect to ClickHouse database and return Client object"""
    client = Client('localhost', settings=SETTINGS)
    client = Client(
        host="10.212.47.162",
        user=os.environ[''],
        password=os.environ[''],
        database=environment
    )
    try: 
        client.connection.connect()
    except ServerException:
        raise ServerException("Connection to ClickHouse failed. Please check your connection information.")
    
    return client

In [9]:
NAME='period_list_weekly_0_0_'
DIRECTORY='period_list_weekly/period_list_weekly'
num=4
date_list=[]

# 
for i in range(num):
    individual_name = f'{NAME}{i}'
    directory = f'C:/Users/tsubasa.ohmura/Downloads/{DIRECTORY}/{individual_name}.csv'
    print(directory)
    if i == 0:
        df = pd.read_csv(directory, names=['COMPANY_CODE', 'STORE_CODE', 'PERIOD'])
    else:
        data = pd.read_csv(directory, names=['COMPANY_CODE', 'STORE_CODE', 'PERIOD'])
        df = df.append(data)
df

C:/Users/tsubasa.ohmura/Downloads/period_list_weekly/period_list_weekly/period_list_weekly_0_0_0.csv
C:/Users/tsubasa.ohmura/Downloads/period_list_weekly/period_list_weekly/period_list_weekly_0_0_1.csv


c:\users\tsubasa.ohmura\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


C:/Users/tsubasa.ohmura/Downloads/period_list_weekly/period_list_weekly/period_list_weekly_0_0_2.csv
C:/Users/tsubasa.ohmura/Downloads/period_list_weekly/period_list_weekly/period_list_weekly_0_0_3.csv


,COMPANY_CODE,STORE_CODE,PERIOD
0,11040,NaN,2019-10-07
1,11040,NaN,2019-10-14
2,11040,NaN,2019-10-21
3,11040,NaN,2019-10-28
4,11040,NaN,2019-11-04
...,...,...,...
358556,900087TK,519501,2021-08-30
358557,900087TK,519501,2021-09-06
358558,900087TK,519501,2021-09-13
358559,900087TK,519501,2021-09-20


In [10]:
df.to_csv(f'C:/Users/tsubasa.ohmura/Downloads/{NAME}.csv', index=False)

In [161]:
# planopos_company_weekly

NAME='company_weekly_0_0 '
DIRECTORY='company_weekly/company_weekly'
NAMES=['COMPANY_CODE', 'STORE_CODE', 'JAN_CODE',
       'PERIOD', 'QUANTITY', 'AMOUNT_OF_MONEY', 'GROSS_PROFIT', 
       'SELLING_PRICE', 'SALE_QUANTITY', 'SALE_AMOUNT', 
       'BARGAIN_GROSS_PROFIT', 'SALE_PRICE', 'NUMBER_OF_DEALERS',
       'NUMBER_OF_VISITORS', 'LOAD_DATE']
num=168
date_list=[]

# 
for i in range(num):
    individual_name = f'{NAME}({i+1})'
    directory = f'C:/Users/tsubasa.ohmura/Downloads/{DIRECTORY}/{individual_name}.csv'
    print(directory)
    if i == 0:
        df = pd.read_csv(directory, names=NAMES)
    else:
        data = pd.read_csv(directory, names=NAMES)
        df = df.append(data)
df = df.astype({'COMPANY_CODE':str, 'STORE_CODE':str, 'JAN_CODE':str, 'NUMBER_OF_VISITORS':float})
df['PERIOD'] = pd.to_datetime(df['PERIOD'])
df['LOAD_DATE'] = pd.to_datetime(df['LOAD_DATE'])
print(df.dtypes)
df

C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (1).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (2).csv


c:\users\tsubasa.ohmura\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (3).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (4).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (5).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (6).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (7).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (8).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (9).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (10).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (11).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (12).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (13)

c:\users\tsubasa.ohmura\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (51).csv


c:\users\tsubasa.ohmura\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (52).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (53).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (54).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (55).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (56).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (57).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (58).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (59).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (60).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (61).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0

C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (141).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (142).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (143).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (144).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (145).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (146).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (147).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (148).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (149).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/company_weekly_0_0 (150).csv
C:/Users/tsubasa.ohmura/Downloads/company_weekly/company_weekly/compan

,COMPANY_CODE,STORE_CODE,JAN_CODE,PERIOD,QUANTITY,AMOUNT_OF_MONEY,GROSS_PROFIT,SELLING_PRICE,SALE_QUANTITY,SALE_AMOUNT,BARGAIN_GROSS_PROFIT,SALE_PRICE,NUMBER_OF_DEALERS,NUMBER_OF_VISITORS,LOAD_DATE
0,3156004,nan,4909411077419,2019-12-16,86.0,16084.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,161268.0,2021-10-25
1,3156004,nan,4909411076658,2019-12-16,482.0,45077.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,227983.0,2021-10-25
2,3156004,nan,4909411071769,2019-12-16,39.0,38937.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,61570.0,2021-10-25
3,3156004,nan,4909411048754,2019-12-16,47.0,8150.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,112899.0,2021-10-25
4,3156004,nan,4909411006297,2019-12-16,10.0,5504.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,51844.0,2021-10-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57527,3110178,nan,4932823037078,2020-03-16,1.0,775.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9255.0,2021-10-18
57528,3110178,nan,4920205506410,2020-03-16,2.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,20379.0,2021-10-18
57529,3110178,nan,49194340,2020-03-16,1.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11444.0,2021-10-18
57530,3110178,nan,4909411085568,2020-03-16,4.0,352.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11812.0,2021-10-18


In [105]:
df.to_csv('C:/Users/tsubasa.ohmura/Downloads/planopos_company_weekly.csv', index=False)

In [75]:
izu = pd.read_csv('C:/Users/tsubasa.ohmura/Downloads/【イズミック】2021年09月_new_v1.csv')
for i in range(len(izu)):
    izu['DELIVERY_DATE'][i] = '20' + str(izu['DELIVERY_DATE'][i])
izu = izu.astype({'WHOLESALER_CODE':str, 'WHOLESALER_CENTRE_CODE':str,
                  'CUSTOMER_BRANCH_CODE':str, 'JAN_CODE':str,
                  'DELIVERY_DATE':str})
izu['JAN_CODE'] = izu['JAN_CODE'].str[0:13]
izu['DELIVERY_DATE'] = pd.to_datetime(izu['DELIVERY_DATE'])

<ipython-input-75-95520ec48fa5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  izu['DELIVERY_DATE'][i] = '20' + str(izu['DELIVERY_DATE'][i])


In [81]:
izu.to_csv('C:/Users/tsubasa.ohmura/Downloads/【イズミック】2021年09月_encoded.csv', index=False)

In [95]:
NAME='store_daily_0 '
DIRECTORY='OneDrive_1_11-9-2021'
NAMES=['COMPANY_CODE', 'STORE_CODE', 'JAN_CODE',
       'PERIOD', 'QUANTITY', 'AMOUNT_OF_MONEY', 'GROSS_PROFIT',
       'SELLING_PRICE', 'SALE_QUANTITY', 'SALE_AMOUNT',
       'BARGAIN_GROSS_PROFIT', 'SALE_PRICE', 'NUMBER_OF_DEALERS',
       'NUMBER_OF_VISITORS', 'LOAD_DATE']
num=149
date_list=[]

# 
for i in range(num):
    individual_name = f'{NAME}({i+1})'
    directory = f'C:/Users/tsubasa.ohmura/Downloads/{DIRECTORY}/{individual_name}.csv'
    print(directory)
    if i == 0:
        df = pd.read_csv(directory, names=NAMES)
    else:
        data = pd.read_csv(directory, names=NAMES)
        df = df.append(data)
df['LOAD_DATE'] = pd.to_datetime(df['LOAD_DATE'])
df

C:/Users/tsubasa.ohmura/Downloads/OneDrive_1_11-9-2021/store_daily_0 (1).csv


FileNotFoundError: [Errno 2] File C:/Users/tsubasa.ohmura/Downloads/OneDrive_1_11-9-2021/store_daily_0 (1).csv does not exist: 'C:/Users/tsubasa.ohmura/Downloads/OneDrive_1_11-9-2021/store_daily_0 (1).csv'

In [91]:
df.to_csv('C:/Users/tsubasa.ohmura/Downloads/planopos_store_test.csv', index=False)

In [97]:
df = pd.read_csv('C:/Users/tsubasa.ohmura/Downloads/planopos_store_test.csv')

In [98]:
df[0:10000].to_csv('C:/Users/tsubasa.ohmura/Downloads/planopos_store_test_small.csv', index=False)

In [163]:
# ClickHouse client execution function.
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

from clickhouse_driver import Client
from clickhouse_driver.errors import ServerException

# Functions
def execute_df_to_ch(clickhouse_client, ch_table, max_fetch_rows, df, \
    planopos_company_weekly=False, truncate=False):
    """Execute sql on HANA database and insert into ClickHouse in max_fetch_rows batches.
    Args:
        clickhouse (clickhouse_driver.client.Client): instance of ClickHouse client
        ch_table (str): destination table in ClickHouse
        max_fetch_rows (int): how many rows to fetch from HANA per one batch.
        daily_sales_transform (bool): whether to do data transforms specific to sales_detail HANA table
    Returns:
        dict:
            row_number (int): total number of ingested rows
            truncated (bool): whether ch_table was truncated or not
    """

    ch_insert_sql = f"INSERT INTO {ch_table} VALUES"

    table_truncated = False
    row_number = 0
    start = 0
    end = max_fetch_rows

    while True:
        print(f"Fetching {row_number}~ rows...")
        result = df[start:end]
        print(f"dataframe fetched from {start}~{end} rows...")
        
        if len(result) == 0 and row_number == 0: raise Exception("Failed to fetch any data.")
        
        if planopos_company_weekly:
            payload = result.values.tolist()
        else:
            payload = result

        print(f"Inserting {len(payload)} rows into ClickHouse {ch_table}.")
        clickhouse_client.execute(ch_insert_sql, payload, settings=SETTINGS)
        row_number += len(payload)
        start += len(payload)
        end += len(payload)
        if len(payload) < max_fetch_rows:
            print("Reached the end of query result set.")
            print(f"Total number of rows: {row_number}")
            break
    
    return {"row_number": row_number, "truncated": table_truncated}

def get_clickhouse_client(environment="commercial"):
    """Connect to ClickHouse database and return Client object"""
    client = Client('localhost', settings=SETTINGS)
    client = Client(
        host="10.212.47.162",
        user=os.environ['CH_USER'],
        password=os.environ['CH_PASS'],
        database=environment
    )
    try: 
        client.connection.connect()
    except ServerException:
        raise ServerException("Connection to ClickHouse failed. Please check your connection information.")
    
    return client



# Define arguments
JOB_NAME = "pos_company_weekly_to_ch"
MAX_FETCH_ROWS = 500_000  # 100,000 rows ~= 650MB of memory
OUT_TABLE = "commercial.planopos_transaction_company_weekly"
SETTINGS={'max_partitions_per_insert_block': 10000}


def main():
    start_time = datetime.now()
    try:
        clickhouse = get_clickhouse_client()
        result = execute_df_to_ch(
            clickhouse_client=clickhouse,
            ch_table=OUT_TABLE,
            max_fetch_rows=MAX_FETCH_ROWS,
            df=df,
            planopos_company_weekly=True
        )
        end_time = datetime.now()
        print(f"Elapsed time: {str(end_time - start_time)}")
    except Exception as e:
        raise e
    finally:
        print('job ended!')

if __name__ == "__main__":
    main()

Fetching 0~ rows...
dataframe fetched from 0~500000 rows...
Inserting 500000 rows into ClickHouse commercial.planopos_transaction_company_weekly.
Fetching 500000~ rows...
dataframe fetched from 500000~1000000 rows...
Inserting 500000 rows into ClickHouse commercial.planopos_transaction_company_weekly.
Fetching 1000000~ rows...
dataframe fetched from 1000000~2000000 rows...
Inserting 1000000 rows into ClickHouse commercial.planopos_transaction_company_weekly.
Fetching 2000000~ rows...
dataframe fetched from 2000000~4000000 rows...
Inserting 2000000 rows into ClickHouse commercial.planopos_transaction_company_weekly.
Fetching 4000000~ rows...
dataframe fetched from 4000000~8000000 rows...
Inserting 4000000 rows into ClickHouse commercial.planopos_transaction_company_weekly.
Fetching 8000000~ rows...
dataframe fetched from 8000000~16000000 rows...
Inserting 8000000 rows into ClickHouse commercial.planopos_transaction_company_weekly.
Fetching 16000000~ rows...
dataframe fetched from 160000

In [168]:
df = pd.read_csv('C:/Users/tsubasa.ohmura/Downloads/goods_master (1).csv')
df.shape

(189593, 37)